In [3]:
import pandas as pd 

file_path = 'Indian_Food_Nutrition_Processed.csv'
food = pd.read_csv(file_path)

foodfinal = food.drop('Vitamin_C', axis=1).drop('Folate', axis=1)
foodfinal.head()
# print(food.columns) to print only column names

,Meal_ID,Dish_Name,Calories,Carbohydrates,Protein,Fats,Free_Sugar,Fibre,Sodium,Calcium,Iron
0,1,Hot tea (Garam Chai),16.14,2.58,0.39,0.53,2.58,0.00,3.12,14.20,0.02
1,2,Instant coffee,23.16,3.65,0.64,0.75,3.62,0.00,4.92,20.87,0.06
2,3,Espreso coffee,51.54,6.62,1.75,2.14,6.53,0.00,13.98,58.10,0.15
3,4,Iced tea,10.34,2.70,0.03,0.01,2.70,0.00,0.23,1.18,0.02
4,5,Raw mango drink (Aam panna),35.92,9.05,0.16,0.03,7.49,0.61,79.82,7.08,0.14


In [2]:
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline

In [ ]:
# Normalize Numerical Features
scaler = StandardScaler()
X_numerical = scaler.fit_transform(foodfinal[['Calories', 'Carbohydrates', 'Protein', 'Fats', 'Free_Sugar', 'Fibre', 'Sodium','Calcium','Iron']])

In [4]:
# Train KNN Model
knn = NearestNeighbors(n_neighbors=3, metric='euclidean')
knn.fit(X_numerical)

,n_neighbors,3
,radius,1.0
,algorithm,'auto'
,leaf_size,30
,metric,'euclidean'
,p,2
,metric_params,None
,n_jobs,None


In [5]:
# Function to Recommend Recipes
def recommend_recipes(input_features):
    input_features_scaled = scaler.transform([input_features[:7]])
    distances, indices = knn.kneighbors(input_features_scaled)
    recommendations = foodfinal.iloc[indices[0]]
    return recommendations[['Dish_Name','Calories', 'Carbohydrates', 'Protein', 'Fats', 'Free_Sugar', 'Fibre', 'Sodium']]

# Example Input
input_features = [50, 5, 2, 10, 5, 0, 2]
recommendations = recommend_recipes(input_features)
recommendations

C:\Users\Rakshith\AppData\Roaming\Python\Python313\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


,Dish_Name,Calories,Carbohydrates,Protein,Fats,Free_Sugar,Fibre,Sodium
2,Espreso coffee,51.54,6.62,1.75,2.14,6.53,0.00,13.98
17,Orange milkshake (Narangi milkshake),57.42,7.11,1.86,2.52,7.08,0.00,14.63
15,Mango milkshake (Aam milkshake),56.90,7.23,1.73,2.35,7.15,0.26,13.27


In [6]:
from pulp import *
import pandas as pd


nutrient_constraints = {
    'Sodium': {'max': 2000},    # mg
    'Fibre': {'max':30}                                   
}

def generate_meal_plan_lp(meals_df, daily_calories, max_meals=3, nutrient_constraints=nutrient_constraints):
    # Create the problem
    prob = LpProblem("MealPlanning", LpMinimize)

    # Macronutrient targets
    macronutrient_ratios = {'carbs': 0.6, 'protein': 0.15, 'fat': 0.25}
    carb_target = (daily_calories * macronutrient_ratios['carbs']) / 4
    protein_target = (daily_calories * macronutrient_ratios['protein']) / 4
    fat_target = (daily_calories * macronutrient_ratios['fat']) / 9

    # Create a binary variable for each meal
    meal_vars = LpVariable.dicts("Meal", meals_df.index, cat="Binary")

    # Objective: minimize deviation from target macros (soft objective)
    prob += lpSum([
        (meals_df.loc[i, 'Calories'] - daily_calories / max_meals) ** 2 * meal_vars[i] +
        (meals_df.loc[i, 'Carbohydrates'] - carb_target / max_meals) ** 2 * meal_vars[i] +
        (meals_df.loc[i, 'Protein'] - protein_target / max_meals) ** 2 * meal_vars[i] +
        (meals_df.loc[i, 'Fats'] - fat_target / max_meals) ** 2 * meal_vars[i]
        for i in meals_df.index
    ])

    # Constraint: total meals selected ≤ max_meals
    prob += lpSum([meal_vars[i] for i in meals_df.index]) == max_meals

    # Caloric constraints (95%-105%)
    total_calories = lpSum([meals_df.loc[i, 'Calories'] * meal_vars[i] for i in meals_df.index])
    prob += total_calories >= daily_calories * 0.95
    prob += total_calories <= daily_calories * 1.05

    # Optional: Macronutrient range constraints (±10%)
    total_carbs = lpSum([meals_df.loc[i, 'Carbohydrates'] * meal_vars[i] for i in meals_df.index])
    total_protein = lpSum([meals_df.loc[i, 'Protein'] * meal_vars[i] for i in meals_df.index])
    total_fat = lpSum([meals_df.loc[i, 'Fats'] * meal_vars[i] for i in meals_df.index])

    prob += total_carbs >= carb_target * 0.9
    prob += total_carbs <= carb_target * 1.1

    prob += total_protein >= protein_target * 0.9
    prob += total_protein <= protein_target * 1.1

    prob += total_fat >= fat_target * 0.9
    prob += total_fat <= fat_target * 1.1

    if nutrient_constraints:
        for nutrient, limits in nutrient_constraints.items():
            total_nutrient = lpSum(meals_df.loc[i, nutrient] * meal_vars[i] for i in meals_df.index)
            if 'min' in limits:
                prob += total_nutrient >= limits['min'], f"{nutrient}_min"
            if 'max' in limits:
                prob += total_nutrient <= limits['max'], f"{nutrient}_max"
                
    # Solve the problem
    prob.solve()
    if LpStatus[prob.status] != 'Optimal':
        print("No feasible solution found with current constraints.")
    # Get selected meals
    selected_meals = meals_df.loc[[i for i in meals_df.index if meal_vars[i].value() == 1]]
    #assert len(selected_meals) == 3, f"Expected 3 meals, got {len(selected_meals)}"
    return selected_meals

selected = generate_meal_plan_lp(foodfinal, 2405.0, max_meals=5, nutrient_constraints=nutrient_constraints)
print(selected)
print()

No feasible solution found with current constraints.
     Meal_ID                                          Dish_Name  Calories  \
318      319                     Gram flour ladoo (Besan ladoo)    476.91   
423      424  Wheat flour and moong dal burfi (Atta aur moon...    441.08   
502      503                                         Plain dosa    380.91   
683      684                                 Gun powder chutney    312.34   
748      749     Pearl millet infant food (Bajra shishu aahaar)    362.44   
749      750                                             Sajina    345.26   
755      756  Rice dal porridge (Chawal dal ki khichdi/khichri)    383.33   
802      803                                         Oats burfi    425.19   
930      931                                  Maa chaane ki dal    344.67   
955      956                              Channa murmura premix    339.09   
956      957  Cracked wheat and green gram dal premix (Dalia...    362.16   
957      958           

In [8]:
target_calories = 839.33+838.46+826.02
print(target_calories)
target_carbs = 12.41
target_protein = 6.8
target_fat = 269.43

# Convert macro targets into ratios
total_macros = target_carbs * 4 + target_protein * 4 + target_fat * 9
macro_ratios = {
    'carbs': (target_carbs * 4) / total_macros,
    'protein': (target_protein * 4) / total_macros,
    'fat': (target_fat * 9) / total_macros
}
print(macro_ratios)

2503.81
{'carbs': 0.019842427779398892, 'protein': 0.010872563166793912, 'fat': 0.9692850090538071}
